In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
!pip install matplotlib
!pip install seaborn

  Using cached seaborn-0.13.1-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.1-py3-none-any.whl (294 kB)


In [7]:
!pip install mlflow


  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached databricks_cli-0.18.0-py2.py3-none-any.whl.metadata (4.0 kB)
  Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)
  Using cached sqlparse-0.4.4-py3-none-any.whl (41 kB)
  Using cached docker-6.1.3-py3-none-any.whl.metadata (3.5 kB)
  Using cached flask-3.0.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached waitress-2.1.2-py3-none-any.whl (57 kB)
  Using cached Mako-1.3.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached werkzeug-3.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from surprise.model_selection import cross_validate,train_test_split,GridSearchCV
from surprise import Dataset,SVD,Reader,accuracy
import mlflow

import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\master-shore-411113-41df45dfb65e.json'

<>:11: SyntaxWarning: invalid escape sequence '\p'
<>:11: SyntaxWarning: invalid escape sequence '\p'
C:\Users\tezi2\AppData\Local\Temp\ipykernel_9660\3714734616.py:11: SyntaxWarning: invalid escape sequence '\p'
  os.environ['GOOGLE_APPLICATION_CREDENTIALS']='D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\master-shore-411113-41df45dfb65e.json'


In [3]:
ratings = pd.read_csv(r'D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\ml-latest-small\ml-latest-small\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movie_meta = pd.read_csv(r'D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\ml-latest-small\ml-latest-small\movies.csv')
movie_meta.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_movie_title = pd.merge(left=ratings,right = movie_meta,how='left',left_on = 'movieId',right_on='movieId')
ratings_movie_title.drop(['timestamp','genres'],inplace=True,axis=1)
ratings_movie_title.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


In [6]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy


In [7]:
# Pivot the data to get a user-item matrix
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings_movie_title[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [8]:
# Create CoClustering model
co_clustering_model = CoClustering(n_cltr_u=3, n_cltr_i=3, n_epochs=20, random_state=None, verbose=False)


In [9]:
# Fit the model on the training set
co_clustering_model.fit(trainset)

In [10]:
# Make predictions on the test set
predictions = co_clustering_model.test(testset)


In [11]:
# Evaluate the model's performance
rmse=accuracy.rmse(predictions)


RMSE: 0.9537


In [12]:
# You can now use the trained model to make recommendations for a specific user
user_id_to_predict = 187
user_ratings = ratings_movie_title[ratings_movie_title['userId'] == user_id_to_predict]
user_unrated_movies = movie_meta[~movie_meta['movieId'].isin(user_ratings['movieId'])]


In [13]:
# Create a list of tuples (movieId, predicted_rating) for unrated movies
unrated_movie_predictions = [(movie_id, co_clustering_model.predict(user_id_to_predict, movie_id).est) for movie_id in user_unrated_movies['movieId']]


In [14]:
# Sort the list by predicted ratings in descending order
sorted_predictions = sorted(unrated_movie_predictions, key=lambda x: x[1], reverse=True)

# Print top N recommendations
top_n = 5
top_recommendations = sorted_predictions[:top_n]
for movie_id, predicted_rating in top_recommendations:
    movie_title = movie_meta[movie_meta['movieId'] == movie_id]['title'].values[0]
    print(f"Movie: {movie_title}, Predicted Rating: {predicted_rating}")

Movie: Heidi Fleiss: Hollywood Madam (1995), Predicted Rating: 5
Movie: Awfully Big Adventure, An (1995), Predicted Rating: 5
Movie: Live Nude Girls (1995), Predicted Rating: 5
Movie: In the Realm of the Senses (Ai no corrida) (1976), Predicted Rating: 5
Movie: Orlando (1992), Predicted Rating: 5


In [15]:
#mlflow
mlflow.set_tracking_uri('http://34.100.213.14:5000/')

exp_name = 'Movie_recommender_system'

#Creating exp_name in mlflow server
if mlflow.get_experiment_by_name(exp_name) is None:
    mlflow.create_experiment(exp_name)
else:
    mlflow.set_experiment(exp_name)

#creating run for coclustering

with mlflow.start_run():
    # Log parameters
    mlflow.log_param('n_cltr_u', 3)
    mlflow.log_param('n_cltr_i', 3)
    mlflow.log_param('n_epochs', 20)
    
    # Log metrics
    mlflow.log_metric('rmse', rmse)
    
    # Log the CoClustering model
    mlflow.sklearn.log_model(co_clustering_model, 'CoClustering_Model')

In [31]:
!pip install google-auth


  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
     ---------------------------------------- 0.0/181.3 kB ? eta -:--:--
     ------ ------------------------------ 30.7/181.3 kB 640.0 kB/s eta 0:00:01
     ------------------------------------ - 174.1/181.3 kB 2.1 MB/s eta 0:00:01
     -------------------------------------- 181.3/181.3 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/186.5 kB ? eta -:--:--
   ------------------------ --------------- 112.6/186.5 kB ? eta -:--:--
   ---------------------------------------- 186.5/186.5 kB 2.8 MB/s eta 0:00:00
Using cached cachetools-5.3.2-py3-none-any.whl (9.3 kB)
   ---------------------------------------- 0.0/84.9 kB ? eta -:--:--
   ---------------------------------------- 84.9/84.9 kB 4.7 MB/s eta 0:00:00


In [33]:
!pip install google-cloud-storage


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/121.6 kB ? eta -:--:--
   ------------- ------------------------- 41.0/121.6 kB 960.0 kB/s eta 0:00:01
   ------------------------------------- -- 112.6/121.6 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 121.6/121.6 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/122.0 kB ? eta -:--:--
   ------------- ------------------------- 41.0/122.0 kB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 122.0/122.0 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/80.6 kB ? eta -:--:--
   -------------------- ------------------- 41.0/80.6 kB 9

In [3]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy
import mlflow
import os
import pickle

def train_and_save_model():
     
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']='D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\master-shore-411113-41df45dfb65e.json'
    # Load the ratings data
    ratings = pd.read_csv(r'D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\ml-latest-small\ml-latest-small\ratings.csv')

    # Load the movie metadata
    movie_meta = pd.read_csv(r'D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\ml-latest-small\ml-latest-small\movies.csv')

    # Merge ratings with movie metadata
    ratings_movie_title = pd.merge(left=ratings, right=movie_meta, how='left', left_on='movieId', right_on='movieId')
    ratings_movie_title.drop(['timestamp', 'genres'], inplace=True, axis=1)

    # Create Surprise dataset
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(ratings_movie_title[['userId', 'movieId', 'rating']], reader)

    # Split the data into training and test sets
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Create CoClustering model
    co_clustering_model = CoClustering(n_cltr_u=3, n_cltr_i=3, n_epochs=20, random_state=None, verbose=False)

    # Fit the model on the training set
    co_clustering_model.fit(trainset)

    # Specify the absolute path to save the pickle file
    pickle_file_path = r'D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\CoClustering_Model.pkl'

    # Save the CoClustering model as a pickle file
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(co_clustering_model, file)

    # Make predictions on the test set
    predictions = co_clustering_model.test(testset)

    # Evaluate the model's performance
    rmse = accuracy.rmse(predictions)

    # Log metrics and model with MLflow
    mlflow.set_tracking_uri('http://34.100.213.14:5000/')
    exp_name = 'Movie_recommender_system'

    # Creating experiment in MLflow
    if mlflow.get_experiment_by_name(exp_name) is None:
        mlflow.create_experiment(exp_name)
    else:
        mlflow.set_experiment(exp_name)

    # Creating run for CoClustering
    with mlflow.start_run():
        # Log parameters
        mlflow.log_param('n_cltr_u', 3)
        mlflow.log_param('n_cltr_i', 3)
        mlflow.log_param('n_epochs', 20)

        # Log metrics
        mlflow.log_metric('rmse', rmse)

        # Log the CoClustering model
        mlflow.sklearn.log_model(co_clustering_model, 'CoClustering_Model')

if __name__ == "__main__":
    train_and_save_model()

<>:12: SyntaxWarning: invalid escape sequence '\p'
<>:12: SyntaxWarning: invalid escape sequence '\p'
C:\Users\tezi2\AppData\Local\Temp\ipykernel_17824\593946147.py:12: SyntaxWarning: invalid escape sequence '\p'
  os.environ['GOOGLE_APPLICATION_CREDENTIALS']='D:\praxis july 24\mlops\ASSIGNMENT\my assignmenrt\MLOPS_project\master-shore-411113-41df45dfb65e.json'


RMSE: 0.9563
